In [ ]:
# use for lora that has not route 
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import re
import sys
import copy
import json
import fire
import torch
import argparse
from tqdm import tqdm
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer, AutoModelForCausalLM, AutoTokenizer
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
model_base = ''
lora_model = ''

In [ ]:
# # 记录gpu显存
# import torch
# torch.cuda.memory._record_memory_history()               # 开始记录
# # run_your_code()                                          # 训练或推理代码
# torch.cuda.memory._dump_snapshot("my_snapshot.pickle")   # 保存文件
# torch.cuda.memory._record_memory_history(enabled=None)   # 终止记录
# torch.cuda.list_gpu_processes()

In [ ]:
# 辅助函数
def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    return

def load_data(dataset) -> list:
    """
    read data from dataset file
    Args:
        args:

    Returns:

    """
    file_path = f'/home/xjz/proj/Subspace-Tuning/CR_MR/dataset/{dataset}/test.json'
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"can not find dataset file : {file_path}")
    json_data = json.load(open(file_path, 'r'))
    return json_data
def extract_answer(dataset, sentence: str) -> float:
    if dataset == 'boolq':
        sentence_ = sentence.strip()
        pred_answers = re.findall(r'true|false', sentence_)
        if not pred_answers:
            return ""
        return pred_answers[0]
    elif dataset == 'piqa':
        sentence_ = sentence.strip()
        pred_answers = re.findall(r'solution1|solution2', sentence_)
        if not pred_answers:
            return ""
        return pred_answers[0]
    elif dataset in ['social_i_qa', 'ARC-Challenge', 'ARC-Easy', 'openbookqa']:
        sentence_ = sentence.strip()
        pred_answers = re.findall(r'answer1|answer2|answer3|answer4|answer5', sentence_)
        if not pred_answers:
            return ""
        return pred_answers[0]
    elif dataset == 'hellaswag':
        sentence_ = sentence.strip()
        pred_answers = re.findall(r'ending1|ending2|ending3|ending4', sentence_)
        if not pred_answers:
            return ""
        return pred_answers[0]
    elif dataset == 'winogrande':
        sentence_ = sentence.strip()
        pred_answers = re.findall(r'option1|option2', sentence_)
        if not pred_answers:
            return ""
        return pred_answers[0]

In [ ]:

# 加载模型 
tokenizer = AutoTokenizer.from_pretrained(model_base)
llm = LLM(
        model = model_base,
        tensor_parallel_size=1,
        gpu_memory_utilization=0.85,
        trust_remote_code=True,
        dtype="half", 
        enforce_eager=False,
        max_model_len=512,
        enable_lora=True, #使用lora微调
        max_lora_rank=32, #lora默认16，大的话要改
    )



In [ ]:
# 自定义一个推理
# 针对每个数据集 包装一个函数
file_path = "/home/xjz/proj/LLaMA-Factory/data/test_set.json"
datasets = json.load(open(file_path, 'r'))
texts = []
output_data = []
correct = 0

for i in range(len(datasets)):
    messages = [
    {"role": "system", "content": datasets[i]['instruction']},
    {"role": "user", "content": datasets[i]['input']},
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    texts.append(text)
# 推理
sampling_params = SamplingParams(temperature=0.7, top_p=0.8,top_k=10,repetition_penalty=1.05, max_tokens=4)
outputs = llm.generate(texts, sampling_params,
                    lora_request=LoRARequest("sui_bian_xie", 1, lora_model))




In [ ]:
correct  =0
allnum = 0
for i,output in enumerate(outputs):
    prompt = output.prompt
    generated_text = output.outputs[0].text
    # print(generated_text)
    # if(len(generated_text) >1):
        # allnum+=1
    predict = generated_text[0]
    label = datasets[i]['output']
    if label == predict:
        correct+=1
    # else :
        # print("predict")
        # print(generated_text)
        # print("label")
        # print(label)
print(correct/len(outputs))

